In [2]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')
final_cleaned_dataset_df=pd.read_csv('final_cleaned_dataset_df.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))
final_cleaned_dataset_df['genres'] = final_cleaned_dataset_df['genres'].apply(lambda x: list(ast.literal_eval(x)))

In [3]:
#Top 5 genres in the dataset is Comedy, Action, Other Fantasy and Adventure

#display(final_cleaned_dataset_df)
mlb=MultiLabelBinarizer()
y_true=mlb.fit_transform(final_cleaned_dataset_df['genres'])

#print("Genre Labels:", mlb.classes_)

# Step 1: Find index of 'Comedy'
comedy_index = list(mlb.classes_).index('Comedy')

# Step 2: Create y_pred with all samples tagged as 'Comedy'
# Same shape as y_true, but only 'Comedy' column is 1
y_pred = np.zeros_like(y_true)
y_pred[:, comedy_index] = 1

print_report(y_true,y_pred,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_true, y_pred, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_true, y_pred)
print("Hit Rate:", hr)


samples avg f1-score = 0.1941602638199497
Jaccard Similarity: 0.14171082238490618
Hit Rate: 0.3862129144851658


c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [4]:
# Define top genres
top1 = ['Comedy']
top2 = ['Comedy', 'Action']
top3 = ['Comedy', 'Action', 'Other']

# Helper to create y_pred based on selected genres
def create_y_pred(genre_list, mlb, y_true):
    indices = [list(mlb.classes_).index(genre) for genre in genre_list]
    y_pred = np.zeros_like(y_true)
    y_pred[:, indices] = 1
    return y_pred

# Helper to compute metrics
def evaluate(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='samples')
    jaccard = jaccard_score(y_true, y_pred, average='samples')
    hit_rate = (np.logical_and(y_true, y_pred).sum(axis=1) > 0).mean()
    return f1, jaccard, hit_rate

# Run evaluations
for label_set, name in zip([top1, top2, top3], ["Top-1", "Top-2", "Top-3"]):
    y_pred = create_y_pred(label_set, mlb, y_true)
    f1, jaccard, hit_rate = evaluate(y_true, y_pred)
    print(f"\n{name} Genres: {label_set}")
    print(f"F1 Score:      {f1:.4f}")
    print(f"Jaccard Index: {jaccard:.4f}")
    print(f"Hit Rate:      {hit_rate*100:.2f}%")


Top-1 Genres: ['Comedy']
F1 Score:      0.1942
Jaccard Index: 0.1417
Hit Rate:      38.62%

Top-2 Genres: ['Comedy', 'Action']
F1 Score:      0.2416
Jaccard Index: 0.1587
Hit Rate:      57.40%

Top-3 Genres: ['Comedy', 'Action', 'Other']
F1 Score:      0.2680
Jaccard Index: 0.1744
Hit Rate:      66.47%


In [5]:
# Define top N genres
top_genres = ['Comedy', 'Action', 'Other', 'Fantasy', 'Adventure']

# Helper: create y_pred from selected top genres
def create_y_pred(genre_list, mlb, y_true):
    indices = [list(mlb.classes_).index(genre) for genre in genre_list]
    y_pred = np.zeros_like(y_true)
    y_pred[:, indices] = 1
    return y_pred

# Helper: calculate metrics
def evaluate(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='samples')
    jaccard = jaccard_score(y_true, y_pred, average='samples')
    hit_rate = (np.logical_and(y_true, y_pred).sum(axis=1) > 0).mean()
    return f1, jaccard, hit_rate

# Loop through Top-1 to Top-5
for n in range(1, 6):
    selected_genres = top_genres[:n]
    y_pred = create_y_pred(selected_genres, mlb, y_true)
    f1, jaccard, hit_rate = evaluate(y_true, y_pred)
    
    print(f"\nTop-{n} Genres: {selected_genres}")
    print(f"F1 Score:      {f1:.4f}")
    print(f"Jaccard Index: {jaccard:.4f}")
    print(f"Hit Rate:      {hit_rate*100:.2f}%")


Top-1 Genres: ['Comedy']
F1 Score:      0.1942
Jaccard Index: 0.1417
Hit Rate:      38.62%

Top-2 Genres: ['Comedy', 'Action']
F1 Score:      0.2416
Jaccard Index: 0.1587
Hit Rate:      57.40%

Top-3 Genres: ['Comedy', 'Action', 'Other']
F1 Score:      0.2680
Jaccard Index: 0.1744
Hit Rate:      66.47%

Top-4 Genres: ['Comedy', 'Action', 'Other', 'Fantasy']
F1 Score:      0.2857
Jaccard Index: 0.1857
Hit Rate:      72.90%

Top-5 Genres: ['Comedy', 'Action', 'Other', 'Fantasy', 'Adventure']
F1 Score:      0.2919
Jaccard Index: 0.1918
Hit Rate:      75.43%
